<a href="https://colab.research.google.com/github/Sahil-Singh2002/Solving-PDE-s-using-Deep-Learning-Algorithms/blob/Sahil-Singh2002-patch-1/2D_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
def domain_1(x):
    # Triangle constraints
    in_triangle = (x[0] >= 0) & (x[1] >= 0) & (x[0] + x[1] <= 1)

    # Point must satisfy both the triangle constraints
    return in_triangle

def domain_2(x, w_i, b_i):
    # Triangle constraints
    in_domain_1 = domain_1(x)

    # Hyperplane constraint: w1*x1 + w2*x2 + b => 0
    in_half_space = (w_i.T @ x - b_i >= 0)

    # Point must satisfy both the triangle constraints and the hyperplane constraint
    return in_domain_1 & in_half_space


def domain_3(x, w_j, w_i, b_i, b_j):
    # Triangle constraints
    in_domain_2 = domain_2(x, w_i, b_i)

    # Hyperplane constraint: w1*x1 + w2*x2 + b => 0
    in_half_space = (w_j.T @ x - b_j >= 0)

    # Point must satisfy both the triangle constraints and the hyperplane constraint
    return in_domain_2 & in_half_space


# Step 1: Solve the linear systems for each node
  # Node 1
A1_1 = np.array([[1, 0], [0, 1]])
b1_1 = np.array([0, 0])
x1 = np.linalg.solve(A1_1, b1_1)


  # Node 2
A2_1 = np.array([[1, 0], [1, 1]])
b2_1 = np.array([0, 1])
x2 = np.linalg.solve(A2_1, b2_1)


  # Node 3
A3_1 = np.array([[0, 1], [1, 1]])
b3_1 = np.array([0, 1])
x3 = np.linalg.solve(A3_1, b3_1)


# Store the results in a list of dictionaries
nodes_1 = [
    {"node": 1, "coordinates": x1},
    {"node": 2, "coordinates": x2},
    {"node": 3, "coordinates": x3}
]

print("____________________Step 1 Nodes order and viable____________________")
# Print the list of nodes
for node in nodes_1:
    print(f"Node {node['node']} coordinates: {node['coordinates']}")


#Step 2
print("____________________Step 2 Nodes order and viable____________________")
  #Hypothetical hyperplane adjust the hyper plane dynamically for the NN later.
w_i = np.array([4,6])
b = 3 #This just implies 4*x1 +6*x2 >= 3 your hyperplane

  #Check if the old nodes from nodes_1 are viable for for the new domain

# Filtering nodes based on domain_2 viability
nodes_1_filtered = [node for node in nodes_1 if domain_2(node["coordinates"], w_i, b)]

print("____________________Nodes 1 filtered to new domain____________________")
# Print the filtered nodes
for node in nodes_1_filtered:
    print(f"Node {node['node']} coordinates: {node['coordinates']} is True")


#Solve the linear systems for each node
  # Node 4
A4_2 = np.array([[1, 1], [4, 6]])
b4_2 = np.array([1, 3])
x4 = np.linalg.solve(A4_2, b4_2)


  # Node 5
A5_2 = np.array([[1, 0], [4, 6]])
b5_2 = np.array([0, 3])
x5 = np.linalg.solve(A5_2, b5_2)


  # Node 6
A6_2 = np.array([[0, 1], [4, 6]])
b6_2 = np.array([0, 3])
x6 = np.linalg.solve(A6_2, b6_2)


# Store the results in a list of dictionaries
nodes_2_dummy = [
    {"node": 4, "coordinates": x4},
    {"node": 5, "coordinates": x5},
    {"node": 6, "coordinates": x6}
]

# Filtering nodes based on domain_2 viability
nodes_2 = [node for node in nodes_2_dummy if domain_2(node["coordinates"], w_i, b)]

print("_____All nodes_2_______")
# Print the filtered nodes
for node in nodes_2:
    print(f"Node {node['node']} coordinates: {node['coordinates']} is True")

def euclidean_distance(coord1, coord2):
    """Calculate the Euclidean distance between two coordinates."""
    return np.linalg.norm(coord1 - coord2)

def reorder_nodes(reference_node, nodes_to_sort):
    """
    Reorders the nodes in `nodes_to_sort` such that the first node is the closest
    to `reference_node`, and each subsequent node is closest to the previous one.
    """
    ordered_nodes = []
    remaining_nodes = nodes_to_sort.copy()
    current_reference = reference_node

    while remaining_nodes:
        # Find the closest node to the current reference
        closest_node = min(
            remaining_nodes,
            key=lambda node: euclidean_distance(current_reference, node["coordinates"])
        )
        # Add the closest node to the ordered list and update the reference
        ordered_nodes.append(closest_node)
        remaining_nodes.remove(closest_node)
        current_reference = closest_node["coordinates"]

    return ordered_nodes

# Use the last node from nodes_1_filtered as the starting reference
if nodes_1_filtered:
    last_node_coordinates = nodes_1_filtered[-1]["coordinates"]
    reordered_nodes_2 = reorder_nodes(last_node_coordinates, nodes_2)

    print("____________________Reordered Nodes 2____________________")
    for node in reordered_nodes_2:
        print(f"Node {node['node']} coordinates: {node['coordinates']}")
else:
    print("No nodes in nodes_1_filtered to use as a reference for reordering nodes_2.")

# Merge nodes_1_filtered and reordered_nodes_2
nodes_2_filtered = nodes_1_filtered + reordered_nodes_2

print("____________________Merged Nodes: nodes_2_filtered____________________")
for node in nodes_2_filtered:
    print(f"Node {node['node']} coordinates: {node['coordinates']}")

____________________Step 1 Nodes order and viable____________________
Node 1 coordinates: [0. 0.]
Node 2 coordinates: [0. 1.]
Node 3 coordinates: [1. 0.]
____________________Step 2 Nodes order and viable____________________
____________________Nodes 1 filtered to new domain____________________
Node 2 coordinates: [0. 1.] is True
Node 3 coordinates: [1. 0.] is True
_____All nodes_2_______
Node 5 coordinates: [0.  0.5] is True
Node 6 coordinates: [0.75 0.  ] is True
____________________Reordered Nodes 2____________________
Node 6 coordinates: [0.75 0.  ]
Node 5 coordinates: [0.  0.5]
____________________Merged Nodes: nodes_2_filtered____________________
Node 2 coordinates: [0. 1.]
Node 3 coordinates: [1. 0.]
Node 6 coordinates: [0.75 0.  ]
Node 5 coordinates: [0.  0.5]


In [17]:
# Step 1: Solve the linear systems for each node

boundary_1,b1 = np.array([1,0]), 0
boundary_2,b2 = np.array([0,1]), 0
boundary_3,b3 = np.array([1,1]), 1

A1_1 = np.array([boundary_1, boundary_2])
b1_1 = np.array([b1, b2])
x1 = np.linalg.solve(A1_1, b1_1)

A2_1 = np.array([boundary_1, boundary_3])
b2_1 = np.array([b1, b3])
x2 = np.linalg.solve(A2_1, b2_1)

A3_1 = np.array([boundary_2, boundary_3])
b3_1 = np.array([b2, b3])
x3 = np.linalg.solve(A3_1, b3_1)

# Store the results in a list of dictionaries, including A and b
nodes_1 = [
    {"node": 1, "coordinates": x1, "A": A1_1, "b": b1_1},
    {"node": 2, "coordinates": x2, "A": A2_1, "b": b2_1},
    {"node": 3, "coordinates": x3, "A": A3_1, "b": b3_1},
]

print("____________________Step 1 Nodes order and viable____________________")
for node in nodes_1:
    print(f"Node {node['node']} coordinates: {node['coordinates']}, A: {node['A']}, b: {node['b']}")

# Step 2: Define hyperplane and filter nodes, needs be dynamically changing
w_i = np.array([4, 6])
b_i = 3

nodes_1_filtered = [node for node in nodes_1 if domain_2(node["coordinates"], w_i, b_i)]

print("____________________Nodes 1 filtered to new domain____________________")
for node in nodes_1_filtered:
    print(f"Node {node['node']} coordinates: {node['coordinates']}, A: {node['A']}, b: {node['b']}")

# Solve the linear systems for the new nodes
A4_2 = np.array([boundary_3, w_i])
b4_2 = np.array([b3, b_i])
x4 = np.linalg.solve(A4_2, b4_2)

A5_2 = np.array([boundary_1, w_i])
b5_2 = np.array([b1, b_i])
x5 = np.linalg.solve(A5_2, b5_2)

A6_2 = np.array([boundary_2, w_i])
b6_2 = np.array([b2, b_i])
x6 = np.linalg.solve(A6_2, b6_2)

# Store the results in a list of dictionaries, including A and b
nodes_2_dummy = [
    {"node": 4, "coordinates": x4, "A": A4_2, "b": b4_2},
    {"node": 5, "coordinates": x5, "A": A5_2, "b": b5_2},
    {"node": 6, "coordinates": x6, "A": A6_2, "b": b6_2},
]

# Filter the new nodes based on domain_2
nodes_2 = [node for node in nodes_2_dummy if domain_2(node["coordinates"], w_i, b_i)]

print("_____All nodes_2_______")
for node in nodes_2:
    print(f"Node {node['node']} coordinates: {node['coordinates']}, A: {node['A']}, b: {node['b']}")

# Reorder nodes_2 based on proximity to the last node in nodes_1_filtered
if nodes_1_filtered:
    last_node_coordinates = nodes_1_filtered[-1]["coordinates"]
    reordered_nodes_2 = reorder_nodes(last_node_coordinates, nodes_2)

    print("____________________Reordered Nodes 2____________________")
    for node in reordered_nodes_2:
        print(f"Node {node['node']} coordinates: {node['coordinates']}, A: {node['A']}, b: {node['b']}")
else:
    print("No nodes in nodes_1_filtered to use as a reference for reordering nodes_2.")

# Merge nodes_1_filtered and reordered_nodes_2
nodes_2_filtered = nodes_1_filtered + reordered_nodes_2

print("____________________Merged Nodes: nodes_2_filtered____________________")
for node in nodes_2_filtered:
    print(f"Node {node['node']} coordinates: {node['coordinates']}, A: {node['A']}, b: {node['b']}")

#This will give the neccesary nodes for the

____________________Step 1 Nodes order and viable____________________
Node 1 coordinates: [0. 0.], A: [[1 0]
 [0 1]], b: [0 0]
Node 2 coordinates: [0. 1.], A: [[1 0]
 [1 1]], b: [0 1]
Node 3 coordinates: [1. 0.], A: [[0 1]
 [1 1]], b: [0 1]
____________________Nodes 1 filtered to new domain____________________
Node 2 coordinates: [0. 1.], A: [[1 0]
 [1 1]], b: [0 1]
Node 3 coordinates: [1. 0.], A: [[0 1]
 [1 1]], b: [0 1]
_____All nodes_2_______
Node 5 coordinates: [0.  0.5], A: [[1 0]
 [4 6]], b: [0 3]
Node 6 coordinates: [0.75 0.  ], A: [[0 1]
 [4 6]], b: [0 3]
____________________Reordered Nodes 2____________________
Node 6 coordinates: [0.75 0.  ], A: [[0 1]
 [4 6]], b: [0 3]
Node 5 coordinates: [0.  0.5], A: [[1 0]
 [4 6]], b: [0 3]
____________________Merged Nodes: nodes_2_filtered____________________
Node 2 coordinates: [0. 1.], A: [[1 0]
 [1 1]], b: [0 1]
Node 3 coordinates: [1. 0.], A: [[0 1]
 [1 1]], b: [0 1]
Node 6 coordinates: [0.75 0.  ], A: [[0 1]
 [4 6]], b: [0 3]
Node 5